In [0]:
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from sklearn.externals import joblib
from keras.models import load_model

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Using TensorFlow backend.


In [0]:
# Define a function that can get n elements from dictionary
from itertools import islice
def take(n, iterable):
  return list(islice(iterable, n))

In [0]:
# Load scaler and model
scaler = joblib.load("scaler.save")
model = load_model('model.h5')

In [0]:
dic = {'Time stamp':[],'Prediction':[]}

In [0]:
# The function get the last three latest close price and return in a list
def getLatestThree():
  url = "https://intraday.worldtradingdata.com/api/v1/intraday?symbol=AAPL&interval=1&range=1&api_token=<Replace with your own API token>"
  response = requests.get(url)
  out = response.json()
  items = take(3,out['intraday'].items())
  closes = [item[1]['close'] for item in items]
  return closes

In [0]:
# The function predicts the next-minute close price using pre-trained model and pre-saved scaler
def prediction():
  closes = getLatestThree()
  closes = scaler.transform(np.array(closes).reshape(-1,1))
  closes = np.array([closes])
  closes = np.reshape(closes, (closes.shape[0],1,closes.shape[1]))
  pred = scaler.inverse_transform(model.predict(closes))
  dic['Time stamp'].append(datetime.today().strftime("%B %d, %Y, %H:%M"))
  dic['Prediction'].append(pred[0][0])
  df = pd.DataFrame.from_dict(dic)
  df.to_csv('0218_apple_pred_rnn.csv',index=None, header=True)
  print('Prediction of the next minute: ' + str(pred[0][0]))
  return df

In [0]:
# This will keep retriving data from API every 60 seconds and output the prediction
import time
while True:
  prediction()
  time.sleep(60)

Prediction of the next minute: 316.9432
Prediction of the next minute: 316.79037
Prediction of the next minute: 316.40686
Prediction of the next minute: 315.72043
Prediction of the next minute: 315.74347
Prediction of the next minute: 315.82385
Prediction of the next minute: 315.93216
Prediction of the next minute: 316.4463
Prediction of the next minute: 316.55103
Prediction of the next minute: 316.46478
Prediction of the next minute: 316.8636
Prediction of the next minute: 316.86234
Prediction of the next minute: 317.1758
Prediction of the next minute: 317.17468
Prediction of the next minute: 316.86438
Prediction of the next minute: 316.8533
Prediction of the next minute: 317.17047
Prediction of the next minute: 317.24585
Prediction of the next minute: 317.3747
Prediction of the next minute: 317.4397
Prediction of the next minute: 317.32587
Prediction of the next minute: 317.26672
Prediction of the next minute: 317.3566
Prediction of the next minute: 317.22812
Prediction of the next m

KeyboardInterrupt: ignored